In [1]:
#importing necessary libraries
from pyspark.sql import SQLContext, Window
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import abs, sqrt

In [2]:
#reading in the data
#creating an SQL Context
sqlContext = SQLContext(sc)
##Creating a dataframe by querying data from the csv file loaded in the Cluster
##Querying Apple Close Price data from 2016-2017 by applying the year function on the date column and specifying it to be between 2016 & 2017
df = sqlContext.sql("SELECT date, close FROM aapl__2__1028f_csv WHERE YEAR(date) BETWEEN 2016 AND 2017")

In [3]:
##displaying the dataframe - two columns, data and close price from 2016-2017
df.show()

+-------------------+----------+
 date| close|
+-------------------+----------+
2016-01-04 00:00:00|105.349998|
2016-01-05 00:00:00|102.709999|
2016-01-06 00:00:00|100.699997|
2016-01-07 00:00:00| 96.449997|
2016-01-08 00:00:00| 96.959999|
2016-01-11 00:00:00| 98.529999|
2016-01-12 00:00:00| 99.959999|
2016-01-13 00:00:00| 97.389999|
2016-01-14 00:00:00| 99.519997|
2016-01-15 00:00:00| 97.129997|
2016-01-19 00:00:00| 96.660004|
2016-01-20 00:00:00| 96.790001|
2016-01-21 00:00:00| 96.300003|
2016-01-22 00:00:00|101.419998|
2016-01-25 00:00:00| 99.440002|
2016-01-26 00:00:00| 99.989998|
2016-01-27 00:00:00| 93.419998|
2016-01-28 00:00:00| 94.089996|
2016-01-29 00:00:00| 97.339996|
2016-02-01 00:00:00| 96.43|
+-------------------+----------+
only showing top 20 rows

In [4]:
##QUESTION 1
###Using the window function to sort the date column in ascending order using the Order by function
partitionwindow = Window.orderBy("date")
## Creating time limited features - lag1 & lag2, where lag1 is the close price pushed by one day using the record difference property and lag2 by 2 days
feature1 =lag("close",1).over(partitionwindow)
feature2 =lag("close",2).over(partitionwindow)
##Generating the columns lag1 and lag2 within our dataframe using the withColumn function 
df = df.withColumn("lag1", feature1)
df = df.withColumn("lag2", feature2)
df = df.na.drop()
##Displaying the dataframe
df.show(20)

+-------------------+----------+----------+----------+
 date| close| lag1| lag2|
+-------------------+----------+----------+----------+
2016-01-06 00:00:00|100.699997|102.709999|105.349998|
2016-01-07 00:00:00| 96.449997|100.699997|102.709999|
2016-01-08 00:00:00| 96.959999| 96.449997|100.699997|
2016-01-11 00:00:00| 98.529999| 96.959999| 96.449997|
2016-01-12 00:00:00| 99.959999| 98.529999| 96.959999|
2016-01-13 00:00:00| 97.389999| 99.959999| 98.529999|
2016-01-14 00:00:00| 99.519997| 97.389999| 99.959999|
2016-01-15 00:00:00| 97.129997| 99.519997| 97.389999|
2016-01-19 00:00:00| 96.660004| 97.129997| 99.519997|
2016-01-20 00:00:00| 96.790001| 96.660004| 97.129997|
2016-01-21 00:00:00| 96.300003| 96.790001| 96.660004|
2016-01-22 00:00:00|101.419998| 96.300003| 96.790001|
2016-01-25 00:00:00| 99.440002|101.419998| 96.300003|
2016-01-26 00:00:00| 99.989998| 99.440002|101.419998|
2016-01-27 00:00:00| 93.419998| 99.989998| 99.440002|
2016-01-28 00:00:00| 94.089996| 93.419998| 99.989998|
2016-01-29 00:00:00| 97.339996| 94.089996| 93.419998|
2016-02-01 00:00:00| 96.43| 97.339996| 94.089996|
2016-02-02 00:00:00| 94.480003| 96.43| 97.339996|
2016-02-03 00:00:00| 96.349998| 94.480003| 96.43|
+-------------------+----------+----------+----------+
only showing top 20 rows

In [5]:
##PREPARING THE DATA TO PERFORM MACHINE LEARNING
##Selecting the relavent columns - where lag1 & lag2 are the features and the close price is the label/observed value we are trying to predict
df = df.select('lag1','lag2','close')
##Using the VectorAssembler function to create an array of vectors containing the values in the lag1 and lag2 columns - FEATURES 
vectorAssembler = VectorAssembler(inputCols = ['lag1','lag2'], outputCol = 'features')
##Transforming the Dataframe containing the lag1, lag2 and close price columns by applying the Vector Assembler function on the input columns
df1 = vectorAssembler.transform(df)
##Selecting the relavent columns from our transformed dataframe and st
df2 = df1.select('features','close')
df2.show()

+--------------------+----------+
 features| close|
+--------------------+----------+
[102.709999,105.3...|100.699997|
[100.699997,102.7...| 96.449997|
[96.449997,100.69...| 96.959999|
[96.959999,96.449...| 98.529999|
[98.529999,96.959...| 99.959999|
[99.959999,98.529...| 97.389999|
[97.389999,99.959...| 99.519997|
[99.519997,97.389...| 97.129997|
[97.129997,99.519...| 96.660004|
[96.660004,97.129...| 96.790001|
[96.790001,96.660...| 96.300003|
[96.300003,96.790...|101.419998|
[101.419998,96.30...| 99.440002|
[99.440002,101.41...| 99.989998|
[99.989998,99.440...| 93.419998|
[93.419998,99.989...| 94.089996|
[94.089996,93.419...| 97.339996|
[97.339996,94.089...| 96.43|
 [96.43,97.339996]| 94.480003|
 [94.480003,96.43]| 96.349998|
+--------------------+----------+
only showing top 20 rows

In [6]:
###QUESTION 2
##Doing a random Train Test split, where 70% of the dataframe is randomly selected for training and 30% of the data is randomly selected for testing
splits = df2.randomSplit([0.7, 0.3])
##Storing the 70% data for training in train_df
train_df = splits[0]
##Storing the 30% data for testing in test_df
test_df = splits[1]
##Initiating the LinearRegression model from the SparkMLLib and specifying the features and the observed value/labels
lr = LinearRegression(featuresCol = 'features', labelCol='close')
##Fitting the Linear Regression Model on the training data and storing the model in lr_model
lr_model = lr.fit(train_df)
##Printing the coefficients of the slope of our Linear Regression Model and the Intercept
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))
##Summarizing our model over the training set and printing out two important metrics - R2 and the RMSE
trainingSummary = lr_model.summary
##RMSE is the square root of the variance of the residuals. It indicates the absolute fit of the model to the data–how close the observed data points are to the model's predicted values.
print("Root Mean Squared Error (RMSE) on train data = %g" % trainingSummary.rootMeanSquaredError)
##R2 is "(total variance explained by model) / total variance.” So if it is 100%, the two variables are perfectly correlated, i.e., with no variance at all. 
print("R Squared (R2) on train data = %g" % trainingSummary.r2)
lr_predictions = lr_model.transform(test_df)

Coefficients: [1.032659502255893,-0.03062986260394673]
Intercept: -0.1333491411478786
Root Mean Squared Error (RMSE) on train data = 1.58299
R Squared (R2) on train data = 0.99616

In [7]:
###QUESTION 3
##Getting the predicted Apple Stock price for 2016-2017 in our test date and comparing it to the Observed Apple Stock price for 2016-2017 in the test data
lr_predictions.select("features", "close", "prediction").show()

+--------------------+---------+-----------------+
 features| close| prediction|
+--------------------+---------+-----------------+
[90.339996,92.510...|90.519997|90.32353751126067|
[92.040001,93.400...|93.589996|92.05180325067569|
[92.510002,93.419...|90.339996|92.53654177466282|
[92.720001,93.239...|92.790001|92.75891261274577|
[92.790001,92.720...|93.419998|92.84712621456815|
[93.239998,94.190...|92.720001|93.26679396394712|
[93.400002,96.099...|92.040001| 93.373520699892|
[93.419998,99.989...|94.089996|93.27501959376976|
[93.589996,92.040...|94.400002|93.69407695964604|
[93.639999,93.739...| 95.18|93.69364235820021|
[94.089996,93.419...|97.339996|94.16813759227011|
 [94.830002,97.82]|93.739998| 94.7975403631794|
[95.099998,95.330...|95.910004|95.15262259477458|
[95.529999,94.989...|95.940002|95.60708148920892|
[95.550003,95.910...|96.099998|95.59955915251724|
 [96.43,95.220001]|97.900002|96.52943111361023|
 [96.43,97.339996]|94.480003|96.46449595803918|
[96.449997,100.69...|96.959999|96.38222968112666|
[96.639999,93.989...|98.120003| 96.7839633993169|
[96.660004,97.129...|96.790001| 96.708444014713|
+--------------------+---------+-----------------+
only showing top 20 rows

In [8]:
###QUESTION 4
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="close",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))
##Evaluating our model on the test data and storing the results in test_result
test_result = lr_model.evaluate(test_df)
##Using the evalution metric RMSE on our Model to get the RMSE of the test data
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

R Squared (R2) on test data = 0.996224
Root Mean Squared Error (RMSE) on test data = 1.61969

In [9]:
###BONUS QUESTION
df2.describe().show()
df2 = df2.withColumn('index', f.monotonically_increasing_id())
new_train = df2.sort('index').limit(350)
new_test = df2.sort('index', ascending=False).limit(151)
new_train.count()
new_test.count()
new_train.select(f.min('index').alias('min'), f.max('index').alias('max')).show()
new_test.select(f.min('index').alias('min'), f.max('index').alias('max')).show()

+-------+------------------+
summary| close|
+-------+------------------+
 count| 501|
 mean|127.62568848902191|
 stddev| 25.7860344769813|
 min| 90.339996|
 max| 176.419998|
+-------+------------------+

+---+---+
min|max|
+---+---+
 0|349|
+---+---+

+---+---+
min|max|
+---+---+
350|500|
+---+---+

In [10]:
##Initiating the LinearRegression model from the SparkMLLib and specifying the features and the observed value/labels
lr1 = LinearRegression(featuresCol = 'features', labelCol='close')
##Fitting the Linear Regression Model on the training data and storing the model in lr_model
lr_model1 = lr1.fit(new_train)
##Printing the coefficients of the slope of our Linear Regression Model and the Intercept
print("Coefficients: " + str(lr_model1.coefficients))
print("Intercept: " + str(lr_model1.intercept))
##Summarizing our model over the training set and printing out two important metrics - R2 and the RMSE
trainingSummary1 = lr_model1.summary
##RMSE is 
print("Root Mean Squared Error (RMSE) on train data = %g" % trainingSummary1.rootMeanSquaredError)
##R2 is "(total variance explained by model) / total variance.” So if it is 100%, the two variables are perfectly correlated, i.e., with no variance at all. 
print("R Squared (R2) on train data = %g" % trainingSummary1.r2)
lr_predictions1 = lr_model1.transform(new_test)
lr_predictions1.select("features", "close", "prediction").show()
lr_evaluator1 = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="close",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator1.evaluate(lr_predictions1))
##Evaluating our model on the test data and storing the results in test_result
test_result1 = lr_model1.evaluate(new_test)
##Using the evalution metric RMSE on our Model to get the RMSE of the test data
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result1.rootMeanSquaredError)

Coefficients: [1.0497021033467877,-0.04670693092307409]
Intercept: -0.2014341688800502
Root Mean Squared Error (RMSE) on train data = 1.45062
R Squared (R2) on train data = 0.992977
+--------------------+----------+------------------+
 features| close| prediction|
+--------------------+----------+------------------+
[171.080002,170.6...|169.229996| 171.4134010753746|
[170.600006,170.5...|171.080002| 170.9109494257973|
[170.570007,175.0...|170.600006|170.67208119958372|
[175.009995,175.0...|170.570007|175.33274594201822|
[175.009995,174.3...|175.009995| 175.3635720026512|
[174.350006,174.5...|175.009995| 174.6619064514802|
[174.539993,176.4...|174.350006|174.77352694131872|
[176.419998,173.9...|174.539993| 176.8614039847619|
[173.970001,172.2...|176.419998|174.37137410978397|
[172.220001,172.2...|173.970001|172.53205994226013|
[172.270004,171.6...|172.220001|172.61117147410846|
[171.699997,172.6...|172.270004|171.96752815758376|
[172.669998,169.3...|171.699997| 173.1398732596992|
[169.369995,169.3...|172.669998|169.67818795561146|
[169.320007,169.0...|169.369995| 169.6401951559387|
[169.009995,169.6...|169.320007|169.28534935416667|
[169.639999,169.8...|169.009995|169.93919258230815|
[169.800003,171.0...|169.639999|170.04876545399821|
[171.050003,171.8...|169.800003|171.32352739832245|
[171.850006,169.4...|171.050003|172.27398812346317|
+--------------------+----------+------------------+
only showing top 20 rows

R Squared (R2) on test data = 0.961641
Root Mean Squared Error (RMSE) on test data = 1.88539